# Visualizations of New York City

This notebook contains code for data pulls and visualizations using three datasets from NYC Open Data.

### 1) Building Footprints
This dataset contains polygon data for the footprints of most of the buildings in NYC. We'll conduct an analysis into building age, and into building sizes.

### 2) WiFi Hotspots
Using City of NYC owned wifi hotspots, we'll create a heatmap of wifi coverage for the city.

### 3) Squirrel Census
A fun and novel dataset of squirrel observations in Central Park. We'll analyze where in the park "loud" squirrels were most often observed.

# Setup

In [1]:
from copy import deepcopy
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
from scipy.stats import gaussian_kde
import seaborn as sns
from shapely.geometry import Point, shape, box, Polygon

pd.options.mode.chained_assignment = None

# Building Footprints

In [2]:
# Pull data
api_endpoint = 'https://data.cityofnewyork.us/resource/qb5r-6dgf.json'
limit = 1000  # Number of rows per request
offset = 0   # Starting offset
data_frames = []  # List to hold chunks of data

# Loop to fetch data iteratively
# while offset <= 100000: # uncomment this and comment while True to fetch a
#   sample much faster
while True: # while True will take a long time but gets all the data
    url = f"{api_endpoint}?$limit={limit}&$offset={offset}"
    chunk = pd.read_json(url)
    if chunk.empty:
        break  # Stop the loop when no more data is returned
    data_frames.append(chunk)
    offset += limit

# Concatenate all chunks into a single DataFrame
data = pd.concat(data_frames, ignore_index=True)

# Convert the 'the_geom' column from a dictionary to a Shapely geometry object
data['geometry'] = data['the_geom'].apply(lambda x: shape(x))

# Convert the Pandas DataFrame to a GeoDataFrame
# Create a raw dataframe which we won't make any edits on and we can copy as
#   needed
gdf = gpd.GeoDataFrame(data, geometry='geometry', crs="EPSG:4326")

# Convert 'MultiPolygon' to representative points for visualization
gdf['centroid'] = gdf['geometry'].centroid.to_crs(epsg=3395).centroid.to_crs(epsg=4326)

# Get rid of columns we don't need anymore
keep_cols = ['cnstrct_yr', 'heightroof', 'geometry', 'centroid']
gdf = gdf[keep_cols]

<ipython-input-2-531adf474c8e>:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid.to_crs(epsg=3395).centroid.to_crs(epsg=4326)


## Building Age and Density of Prewars

In [29]:
# Create the bounding box from the provided corner points
bounding_box = box(-74.0294, 40.685, -73.91695, 40.742)

# Create copy of gdf for this analysis
gdf_building_age = deepcopy(gdf)

# Filter the GeoDataFrame using the bounding polygon
gdf_building_age[gdf_building_age['centroid'].within(bounding_box)]

# Create a new column for building decade
gdf_building_age['decade'] = (gdf_building_age['cnstrct_yr'] // 10) * 10

# Remove rows where 'cnstrct_yr' is NaN
gdf_building_age = gdf_building_age[gdf_building_age['cnstrct_yr'].notna()]

# Get unique decades
unique_decades = sorted(gdf_building_age['decade'].unique())

# Use the Cividis colorscale and split it for our unique decades
colorscale = px.colors.sequential.Cividis
num_decades = len(unique_decades)
colors = [colorscale[i * len(colorscale) // num_decades] for i in range(num_decades)]
color_map = dict(zip(unique_decades, colors))

# Filter the data for buildings built in the 1930s and earlier
old_buildings = gdf_building_age[gdf_building_age['decade'] <= 1930]

# Create a new figure for better control
fig = go.Figure()

# Add the traces for each decade
for decade, color in color_map.items():
    subset = gdf_building_age[gdf_building_age['decade'] == decade]
    # Add the original trace with showlegend set to False
    fig.add_trace(go.Scattermapbox(
        lat=subset['centroid'].y,
        lon=subset['centroid'].x,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=3,
            color=color,
            opacity=0.8
        ),
        text=decade,
        name=str(int(decade)),
        hoverinfo='none',
        showlegend=False
    ))
    # Add a dummy trace with larger markers for the legend
    #   placed outside the visible map
    fig.add_trace(go.Scattermapbox(
        lat=[90],  # Latitude outside the visible map
        lon=[0],   # Longitude outside the visible map
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color=color,
            opacity=1
        ),
        legendgroup=str(int(decade)),
        showlegend=True,
        name=str(int(decade)),
        hoverinfo='none'
    ))

# Add heatmap for older buildings
fig.add_trace(go.Densitymapbox(
    lat=old_buildings['centroid'].y,
    lon=old_buildings['centroid'].x,
    radius=4,
    colorscale="Greens",
    opacity=1,
    name="Density of Prewar Buildings",
    showlegend=True,
    zmax=3,
    zmin=0,
    showscale=False
))


fig.update_layout(
    title='Buildings by Decade with Density Underlay for Prewar Buildings',
    autosize=True,
    mapbox=dict(
        accesstoken=None,
        bearing=0,
        center=dict(lat=40.71359, lon=-73.97216),
        pitch=0,
        zoom=12.6,
        style='carto-positron'
    ),
    height=800,
    width=1200,
    legend=dict(tracegroupgap=0)
)

# Display the map
fig.show()

## Average Building Size by Borough

In [25]:
# ...following from data pull code block

# Define borough bounding boxes
# These are very loose bounds and a more thorough analysis should use a higher
#   precision polygon.
boroughs = {
    "Manhattan": box(-74.02, 40.70, -73.91, 40.88),
    "Bronx": box(-73.93, 40.80, -73.79, 40.92),
    "Brooklyn": box(-74.05, 40.57, -73.85, 40.74),
    "Queens": box(-73.94, 40.54, -73.70, 40.80),
    "Staten Island": box(-74.26, 40.50, -74.03, 40.65)
}

# Create a function that assigns a borough to each building based on its
#   centroid
def assign_borough(centroid):
    for borough, bbox in boroughs.items():
        if bbox.contains(centroid):
            return borough
    return None

# Create copy of gdf for this analysis
gdf_building_size = deepcopy(gdf)

# Place each building into a borough
gdf_building_size['borough'] = gdf_building_size['centroid'].apply(assign_borough)

# Calculate building volume using footprint area and height
gdf_building_size['volume'] = gdf_building_size['geometry'].area * gdf_building_size['heightroof']

# Compute average volume by borough
avg_volume_by_borough = gdf_building_size.groupby('borough')['volume'].median()

# Create 3D bar shapes using surface plots
def create_3d_bar(x, y, z, dx, dy, dz):
    # Define vertices of the bar
    x_data = [[x, x, x+dx, x+dx, x], [x, x, x+dx, x+dx, x]]
    y_data = [[y, y+dy, y+dy, y, y], [y, y+dy, y+dy, y, y]]
    z_data = [[z, z, z, z, z], [z+dz, z+dz, z+dz, z+dz, z+dz]]
    return go.Surface(
        x=x_data,
        y=y_data,
        z=z_data,
        colorscale=[[0, 'dodgerblue'], [1, 'dodgerblue']],
        showscale=False,
        opacity=0.5
    )

# Define bar dimensions
dx = 0.6
dy = 0.4

# Create figure
fig = go.Figure()

# Add bars to the figure
for i, borough in enumerate(avg_volume_by_borough.index):
    fig.add_trace(create_3d_bar(i, 0, 0, dx, dy, avg_volume_by_borough[borough]))

# Define the layout with adjusted aspect ratio for wider chart area
fig.update_layout(
    title='Median Building Volume by Borough',
    scene=dict(
        xaxis=dict(
            title='Borough',
            tickvals=list(range(len(avg_volume_by_borough))),
            ticktext=avg_volume_by_borough.index
        ),
        yaxis=dict(title='', visible=False),
        zaxis=dict(title='Average Building Volume (m^3)'),
        aspectratio=dict(x=3.5, y=2, z=1.5)  # Adjusting the aspect ratio for wider x-axis
    ),
    margin=dict(t=40, b=40, l=60, r=60),
    height=800,
    width=1200
)

fig.show()

<ipython-input-25-ebbfe0d62571>:29: UserWarning:

Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
# ...following from 3D bar visual of building volume

# Set figure size
plt.figure(figsize=(12, 8))

# Create the boxenplot
sns.boxenplot(
    data=gdf_building_size,
    y='borough',
    x='volume',
    k_depth='trustworthy',
    order=['Manhattan', 'Bronx', 'Brooklyn', 'Staten Island', 'Queens']
)

# Adjust the x-axis to use a log scale
plt.xscale('log')

# Set title and axis labels
plt.title('Building Size Distribution by Volume')
plt.xlabel('Building Volume (log m^3)')
plt.ylabel('Borough')

plt.show()

## WiFi Hotspots

In [26]:
# Define the URL
url = "https://data.cityofnewyork.us/resource/yjub-udmw.json"

# Send a GET request
response = requests.get(url)

# Load the response into a JSON
data = response.json()

# Convert the JSON data into a DataFrame
df = pd.DataFrame(data)

# Convert lat and lon columns to float
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Map the borough codes to borough names
borough_dict = {'1': 'Manhattan', '2': 'Bronx', '3': 'Brooklyn', '4': 'Queens', '5': 'Staten Island'}
df['borough'] = df['borough'].map(borough_dict)

# Replace the 'token' with your own Mapbox access token
px.set_mapbox_access_token('token')

fig = px.density_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    zoom=10,
    mapbox_style="carto-positron",
    title="Distribution of WiFi Hotspots in NYC",
    radius=6
)

fig.update_layout(
    height=800,
    width=1200
)

fig.show()

## Squirrel Census

In [27]:
# Pull data
data_url = 'https://data.cityofnewyork.us/api/views/vfnx-vebw/rows.csv?accessType=DOWNLOAD'
squirrels = pd.read_csv(
    data_url,
    usecols=['X', 'Y', 'Kuks', 'Quaas', 'Moans']
)

# Create column denoting that the squirrel made any kind of noise
squirrels['noisy'] = squirrels[['Kuks', 'Quaas', 'Moans']].any(axis=1)

# Filter out the quiet squirrels
noisy_squirrels = squirrels[squirrels['noisy']]

# Convert noisy column to integer
noisy_squirrels['noisy'] = noisy_squirrels['noisy'].astype(int)

# Create the density heatmap
fig = px.density_mapbox(
    noisy_squirrels, lat='Y', lon='X', z='noisy', radius=50,
    center=dict(lat=40.783, lon=-73.969),  # Center coordinates for Central Park
    zoom=13,
    mapbox_style="stamen-terrain",
    #mapbox_style="stamen-watercolor",
    color_continuous_scale=["white", "orange", "red"],
    range_color=[0, 5], # Adjusting the range for color scale
)

# Set the bearing to orient Central Park horizontally
fig.update_layout(
    mapbox_bearing=0,
    height=700,
    width=1000,
    title='Density of Noisy Squirrel Observations',
    showlegend=False,
    coloraxis_showscale=False
)

fig.show()